In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
!pip install transformers

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
import logging
from huggingface_hub import notebook_login
from pathlib import Path

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

In [6]:
!pip install accelerate
!pip install auto-gptq
!pip install --upgrade optimum

In [12]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [13]:
def gen(p,maxlen=15, sample=True):
  toks = tokr(p , return_tensors='pt')
  res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to("cpu")
  return tokr.batch_decode(res)

In [10]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [11]:
def mk_pmt(user, syst=sb_sys): return f"{syst}###user: {user}\n\n### Assistant:\n"

In [8]:
ques = "Who is Jeremy Howard?"

In [7]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
#mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

## Retrival Augmented Generation


In [14]:
!pip install Wikipedia-api

In [15]:
from wikipediaapi import Wikipedia

In [16]:
wiki = Wikipedia("JeremyHowardBot/0.0",'en')
jh_pg = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_pg = jh_pg.split('\nReferences\n')[0]

In [17]:
print(jh_pg[:500])

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator.He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.

Early life
Howard was born in London, United Kingdom, and move


In [18]:
len(jh_pg.split())

613

In [19]:
ques_ctx = f"""Answer the question with the help of the provided context.

## Context

{jh_pg}

## Question

{ques}"""

In [20]:
res = gen(mk_pmt(ques_ctx),300)

In [21]:
print(res[0].split('### Assistant:\n')[1])

 Jeremy Howard is an Australian data scientist, entrepreneur, and educator who is known for his contributions in deep learning and research in the field of artificial intelligence. He is the co-founder of fast.ai and has been involved in various startups such as Fastmail, Optimal Decisions Group (ODG), and Enlitic. Throughout his career, Howard has also served as President and Chief Scientist of Kaggle and was recognized as the globally top-ranked participant in data science competitions. Additionally, he teaches data science and has a strong interest in furthering AI and deep learning research through his work with fast.ai.</s>


In [2]:
!pip install -U sentence-transformers

In [22]:
#from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer

In [24]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [26]:
jh = jh_pg.split('\n\n')[0]
print(jh)

Jeremy Howard (born 13 November 1973) is an Australian data scientist, entrepreneur, and educator.He is the co-founder of fast.ai, where he teaches introductory courses, develops software, and conducts research in the area of deep learning.
Previously he founded and led Fastmail, Optimal Decisions Group, and Enlitic. He was President and Chief Scientist of Kaggle.
Early in the COVID-19 epidemic he was a leading advocate for masking.


In [27]:
tb_pg = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [30]:
tb = tb_pg.split('\n\n')[0]
print(tb[:380])

Sir Anthony Charles Lynton Blair  (born 6 May 1953) is a British politician who served as Prime Minister of the United Kingdom from 1997 to 2007 and Leader of the Labour Party from 1994 to 2007. He served as Leader of the Opposition from 1994 to 1997 and had various shadow cabinet posts from 1987 to 1994. Blair was Member of Parliament (MP) for Sedgefield from 1983 to 2007. He 


In [32]:
q_emb, jh_emb, tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [33]:
tb_emb.shape

torch.Size([384])

In [34]:
import torch.nn.functional as F

In [35]:
F.cosine_similarity(q_emb,jh_emb, dim=0)

tensor(0.7991, device='cuda:0')

In [36]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

tensor(0.5315, device='cuda:0')